# Import lib


In [272]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt

# Read data

In [273]:
df = pd.read_csv('../data.csv')

/var/folders/l9/lgcrjc856bb1g071byrjw6rr0000gn/T/ipykernel_76261/3454644713.py:1: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data.csv')


# Discover and visualise the data

#### remove irrelevant features

In [274]:
remove_col = [
    'id',
    'member_id',
    'emp_title',
    'issue_d',
    'desc',
    'title',
    'zip_code',
    'addr_state',
    'earliest_cr_line',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'next_pymnt_d',
    'last_credit_pull_d',
    'collections_12_mths_ex_med',
    'mths_since_last_major_derog',
    'policy_code',

    'purpose', 
    'home_ownership', # contains 'other', 'any'
    'sub_grade',
    'tot_coll_amt', 
    'tot_cur_bal', 
    'open_acc_6m', 
    'open_il_6m', 
    'open_il_12m', 
    'open_il_24m', 
    'mths_since_rcnt_il', 
    'total_bal_il', 
    'il_util', 
    'open_rv_12m' ,
    'open_rv_24m', 
    'max_bal_bc', 
    'all_util', 
    'total_rev_hi_lim', 
    'inq_fi', 
    'total_cu_tl', 
    'inq_last_12m'
]

### Handle home_ownership

In [304]:
df['home_ownership'].unique()

# Values with ANY can be removed since they are noise 


array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE'], dtype=object)

In [303]:
# Drop rows with ANY 

df = df[df['home_ownership'] != 'ANY']


All target class for row with ANY are 0, can remove all row with any for home_ownership

In [276]:
# df = df.drop(remove_col, axis=1)

In [277]:
np.shape(df)

(855969, 73)

In [278]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855969 entries, 0 to 855968
Data columns (total 73 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           855969 non-null  int64  
 1   member_id                    855969 non-null  int64  
 2   loan_amnt                    855969 non-null  int64  
 3   funded_amnt                  855969 non-null  int64  
 4   funded_amnt_inv              855969 non-null  float64
 5   term                         855969 non-null  object 
 6   int_rate                     855969 non-null  float64
 7   installment                  855969 non-null  float64
 8   grade                        855969 non-null  object 
 9   sub_grade                    855969 non-null  object 
 10  emp_title                    806526 non-null  object 
 11  emp_length                   812908 non-null  object 
 12  home_ownership               855969 non-null  object 
 13 

### Process missing value

In [279]:
# Display all attributes with missing values

null_columns = df.columns[df.isnull().any()] 
null_columns_result = df.isnull().any()[null_columns] 
null_columns_result

emp_title                      True
emp_length                     True
desc                           True
title                          True
mths_since_last_delinq         True
mths_since_last_record         True
revol_util                     True
last_pymnt_d                   True
next_pymnt_d                   True
last_credit_pull_d             True
collections_12_mths_ex_med     True
mths_since_last_major_derog    True
annual_inc_joint               True
dti_joint                      True
verification_status_joint      True
tot_coll_amt                   True
tot_cur_bal                    True
open_acc_6m                    True
open_il_6m                     True
open_il_12m                    True
open_il_24m                    True
mths_since_rcnt_il             True
total_bal_il                   True
il_util                        True
open_rv_12m                    True
open_rv_24m                    True
max_bal_bc                     True
all_util                    

#### Emp_length

In [280]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [281]:
label_encoder = LabelEncoder()

# Fit the LabelEncoder with unique values
label_encoder.fit(df['emp_length'].unique())

# Encode the attribute values
df['emp_length'] = label_encoder.transform(df['emp_length'])

In [282]:
df['emp_length'].unique()

array([ 1, 10,  0,  3,  8,  9,  4,  5,  6,  2,  7, 11])

#### mths_since_last_delinq 

In [283]:
df['mths_since_last_delinq'] = df['mths_since_last_delinq'].fillna(-1)

#### mths_since_last_record

In [284]:
df['mths_since_last_record'] = df['mths_since_last_record'].fillna(-1)

#### revol_util

In [285]:
imputer = SimpleImputer(strategy='mean')

revol_util = df['revol_util'].values.reshape(-1,1)

revol_util_imputed = imputer.fit_transform(revol_util)

df['revol_util'] = revol_util_imputed

#### annual_inc_joint

In [286]:
# aggregate annual_inc and annual_inc_joint
df.loc[df['application_type'] == 'JOINT', 'annual_inc'] = df.loc[df['application_type'] == 'JOINT', 'annual_inc_joint']
df = df.drop('annual_inc_joint', axis=1)

#### dti_joint 

In [287]:
df.loc[df['application_type'] == 'JOINT', 'dti'] = df.loc[df['application_type'] == 'JOINT', 'dti_joint']
df = df.drop('dti_joint', axis=1)

#### verification_status_joint 

In [288]:
df.loc[df['application_type'] == 'JOINT', 'verification_status'] = df.loc[df['application_type'] == 'JOINT', 'verification_status_joint']
df = df.drop('verification_status_joint', axis=1)

### Data binning

#### term

In [289]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [290]:
term = [' 36 months', ' 60 months']  # Unique values for encoding

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the grades
encoder.fit(term)

# Encode the 'grade' column in the DataFrame
df['term'] = encoder.transform(df['term'])

### grade

In [291]:
grades = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(grades)

df['grade'] = encoder.transform(df['grade'])

### verification_status

In [292]:
veri = ['Not Verified', 'Source Verified', 'Verified']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(veri)


df['verification_status'] = encoder.transform(df['verification_status'])

### pymnt_plan

In [293]:
plan = ['n', 'y']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(plan)

df['pymnt_plan'] = encoder.transform(df['pymnt_plan'])

### application_type

In [294]:
type = ['INDIVIDUAL', 'JOINT'] # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(type)

df['application_type'] = encoder.transform(df['application_type'])

#### initial_list_status

In [295]:

status = ['f', 'w'] # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(status)

df['initial_list_status'] = encoder.transform(df['initial_list_status'])

### Generate new feature

In [296]:
# Generate the new feature
df['open_acc_rate'] = df['open_acc'] / df['total_acc']
# Swap the values and column names
df['open_acc_rate'], df['default_ind'] = df['default_ind'].copy(), df['open_acc_rate'].copy()
df.rename(columns={'open_acc_rate': 'default_ind', 'default_ind': 'open_acc_rate'}, inplace=True)

In [297]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,open_acc_rate,default_ind
0,1077501,1296599,5000,5000,4975.0,0,10.65,162.87,1,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0
1,1077430,1314167,2500,2500,2500.0,1,15.27,59.83,2,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750000,1
2,1077175,1313524,2400,2400,2400.0,0,15.96,84.33,2,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200000,0
3,1076863,1277178,10000,10000,10000.0,0,13.49,339.31,2,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.270270,0
4,1075358,1311748,3000,3000,3000.0,1,12.69,67.79,1,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.394737,0


In [298]:
np.shape(df)

(855969, 71)

### Visualize

In [299]:
# Create a correlation matrix
corr_matrix = df.corr()

# Select the correlation values with 'default_ind'
target_corr = corr_matrix['default_ind']

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(12, 12))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

ValueError: could not convert string to float: 'B2'

### Attribute ranking

In [ ]:
# Take the absolute values of the correlation
abs_corr = target_corr.abs()

# Sort the absolute correlation values
sorted_corr = abs_corr.sort_values(ascending=False)

# Display the attribute ranking
attribute_ranking = sorted_corr.reset_index()
attribute_ranking.columns = ['Attribute', 'Absolute Correlation']
attribute_ranking

,Attribute,Absolute Correlation
0,default_ind,1.000000
1,recoveries,0.475738
2,collection_recovery_fee,0.330764
3,out_prncp,0.225960
4,out_prncp_inv,0.225959
5,int_rate,0.155037
6,total_rec_late_fee,0.140760
7,grade,0.123656
8,initial_list_status,0.098812
9,total_rec_prncp,0.090336
